# To properly try sharding, run pip install ./zarr_python

This will install the Sharding Pull Request in the zarr pip list

In [1]:
import zarr
zarr.__version__

'2.13.0a2.dev42+dirty'

^^^ Should be '2.13.0a2.dev42+dirty' if Sharding PR is installed

In [2]:
import numpy as np


numpy_arr = np.empty((60000, 15000), dtype=np.uint32) # ~3GB
original_shape = numpy_arr.shape

# 50k x 150k int32 is limit of M1 Pro 32GB
#
for i in range(numpy_arr.shape[0]):
    numpy_arr[i][:] = list(range(original_shape[1]*i, original_shape[1]*(i+1)))

In [3]:
# Let's make chunks really large to test partial decompression

# So Let's use a weak compressor
from numcodecs import Blosc
weak_compressor = Blosc(cname='lz4', clevel=1, shuffle=Blosc.NOSHUFFLE)

chunk_size=(15000,15000)

# Reached max chunk size
zarr_index_list = zarr.array(numpy_arr, compressor=weak_compressor, chunks=chunk_size, partial_decompress=True, dtype='f8')
# z = zarr.zeros((10000,10000), chunks=(1000,1000), dtype='i4')

/opt/homebrew/lib/python3.10/site-packages/zarr/creation.py:241: UserWarning: ignoring keyword argument 'partial_decompress'
  warn('ignoring keyword argument %r' % k)


In [4]:
zarr_index_list.info

Type,zarr.core.Array
Data type,float64
Shape,"(60000, 15000)"
Chunk shape,"(15000, 15000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=1, shuffle=NOSHUFFLE, blocksize=0)"
Store type,zarr.storage.KVStore
No. bytes,7200000000 (6.7G)
No. bytes stored,3630779699 (3.4G)
Storage ratio,2.0


In [ ]:
# Read it properly, without partial_decompressing chunks
load_disk_normally = zarr.open("./example-zarr", mode="r", partial_decompress=False)
load_disk_normally.info